# Get human-estimated cloud cover from CTIO night reports

## Notebook preparation

In [1]:
import urllib
from contextlib import closing
from astropy.time import Time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

## Cloud reports compiled from multiple telescope operators by CTIO staff

CTIO staff provided data (in eighths cloud cover by quarter of night) for dates before 2014-03-30.

In [2]:
!head clouds_1975-01-01_to_2014-03-30.txt

sday	eday	month	year	q1	q2	q3	q4
1	2	1	1975	0	0	0	0
2	3	1	1975	0	2	3	4
3	4	1	1975	0	3	4	0
4	5	1	1975	0	2	3	4
5	6	1	1975	3	2	1	1
6	7	1	1975	0	0	0	0
7	8	1	1975	0	0	0	0
8	9	1	1975	0	0	0	0
9	10	1	1975	0	0	0	0


## Scrape cloud values from the Blanco night reports

Night reports from the Blanco telescope include eighths reported cloud cover by quarter. These are not provided in any compiled way, but must by obtained by querying each nights' report and extracting the relevant field.

In [3]:
def get_night_clouds(time_repr):
    try:
        t = Time(time_repr, format='mjd')
    except ValueError:
        t = Time(time_repr)
    year = t.datetime.year
    month = t.datetime.month
    day = t.datetime.day
    next_day = Time(t.mjd+1, format='mjd').datetime.day

    report_url = f"http://www.ctio.noirlab.edu/noao/night-report-sky-conditions?field_nr_date_value%5Bvalue%5D%5Bdate%5D={month:02d}%2F{day:02d}%2F{year}"
    with closing(urllib.request.urlopen(report_url)) as f:
        report_page = f.read()

    report_soup = BeautifulSoup(report_page)
    
    sky = [day, next_day, month, year]
    for quarter in range(1, 5):
        fields = report_soup.find_all(class_=f'field-name-field-field-nr-{quarter}-sky')
        if len(fields) < 1:
            sky.append(9)
            continue

        assert fields[0].find_all(class_='field-label')[0].text.startswith('Sky')
        values = fields[0].find_all(class_='field-item')
        assert len(values) == 1
        sky.append(int(values[0].text))

    return sky

In [4]:
%%time
begin_date_iso = '2014-03-30'
end_date_iso = '2022-09-04'

scraped_clouds_fname = f'clouds_{begin_date_iso}_to_{end_date_iso}.txt'

if not os.path.exists(scraped_clouds_fname):
    first_mjd = Time(begin_date_ios).mjd.astype(int)
    last_mjd = Time(end_date_iso).mjd.astype(int)
    nights = []

    for mjd in range(first_mjd, last_mjd+1):
        sky = get_night_clouds(mjd)
        nights.append(sky)
        t = Time(mjd, format='mjd')
        if t.datetime.day == 1:
            print()
            print(t.iso, end='')
        else:
            print('.', end='')
    print()

    nights_df = pd.DataFrame(nights, columns=['sday', 'eday', 'month', 'year', 'q1', 'q2', 'q3', 'q4'])
    nights_df.describe()
    nights_df.to_csv(scraped_clouds_fname, sep="\t", index=False)

CPU times: user 85 µs, sys: 34 µs, total: 119 µs
Wall time: 113 µs


In [5]:
2

2